<a href="https://colab.research.google.com/github/MattWroclaw/neural-networks/blob/main/03_keras/03_udnerfitting_overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Główne problemy uczenia maszynowego: przeuczenie (overfitting) oraz niedouczenie (underfitting)

>Celem tego notebook'a jest pokazanie przykładów zbyt dobrego dopasowanie modelu do danych uczących (przeuczenie) oraz zbyt słabego dopasowania modelu do danych uczących (niedouczenie).
>
>Wykorzystamy zbiór z bilioteki Keras składający się z 50000 recenzji filmów oznaczonych sentymentem: pozytywny/negatywny. Recenzje są wstępnie przetworzone, a każda recenzja jest zakodowana jako sekwencja indeksów słów. Słowa są indeksowane według ogólnej częstotliwości w zbiorze danych. Na przykład liczba 5 oznacza piąte najczęściej pojawiające się słowo w danych. Liczba 0 nie oznacza określonego słowa.

### Spis treści
1. [Import bibliotek](#a1)
2. [Załadowanie i przygotowanie danych](#a2)
3. [Budowa modelu bazowego](#a3)
4. [Budowa 'mniejszego' modelu](#a4)    
5. [Budowa 'większego' modelu](#a5)
6. [Porównanie wydajności modeli](#a6)
7. [Metody regularyzacji](#a7)

In [ ]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

!pip uninstall -y tensorflow
!pip install -q tensorflow==2.0.0

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets.imdb import get_word_index
from tensorflow.keras.utils import get_file
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

sns.set()
tf.__version__

'2.17.0'

### <a name='a2'></a> 2. Załadowanie i przygotowanie danych

In [2]:
NUM_WORDS = 10000   # 10000 najczęściej pojawiających się słów
INDEX_FROM = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
print(f'train_data shape: {train_data.shape}')
print(f'test_data shape: {test_data.shape}')

train_data shape: (25000,)
test_data shape: (25000,)


In [4]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [5]:
word_to_idx = get_word_index()
word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
word_to_idx['<PAD>'] = 0
word_to_idx['<START>'] = 1
word_to_idx['<UNK>'] = 2
word_to_idx['<UNUSED>'] = 3
idx_to_word = {v: k for k, v in word_to_idx.items()}
list(idx_to_word.items())[:10]
print(' '.join(idx_to_word[idx] for idx in train_data[0]))


1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these c

Ten kod w Pythonie dotyczy przetwarzania słów na ich indeksy w celu przygotowania danych tekstowych do modelu uczenia maszynowego, np. przy użyciu sekwencji słów w sieciach neuronowych. Wyjaśnię krok po kroku.

### Co robi kod?

1. **Pobranie indeksów słów**:
   ```python
   word_to_idx = get_word_index()
   ```
   - Funkcja `get_word_index()` prawdopodobnie pobiera słownik, w którym kluczem jest słowo, a wartością jego indeks (czyli unikalna liczba przypisana do każdego słowa). Przykładowo, słowo "apple" może być mapowane na indeks 25.

2. **Modyfikacja indeksów słów**:
   ```python
   word_to_idx = {k:(v + INDEX_FROM) for k, v in word_to_idx.items()}
   ```
   - To przekształcenie przesuwa wszystkie wartości indeksów o pewną liczbę `INDEX_FROM`. Załóżmy, że `INDEX_FROM = 3`, wtedy wszystkie indeksy zostaną powiększone o 3.
   - Powód tego przesunięcia to często zarezerwowanie początkowych indeksów dla specjalnych tokenów, jak `'<PAD>'`, `'<START>'`, itp.

3. **Dodanie specjalnych tokenów**:
   ```python
   word_to_idx['<PAD>'] = 0
   word_to_idx['<START>'] = 1
   word_to_idx['<UNK>'] = 2
   word_to_idx['<UNUSED>'] = 3
   ```
   - Tokeny specjalne są dodawane ręcznie do słownika:
     - `'<PAD>' = 0`: Reprezentuje padding, używany do wyrównania sekwencji do tej samej długości.
     - `'<START>' = 1`: Wskazuje początek sekwencji.
     - `'<UNK>' = 2`: Oznacza nieznane słowo (ang. unknown).
     - `'<UNUSED>' = 3`: Rezerwowy token, często nieużywany w praktyce, ale zarezerwowany.

4. **Odwrócenie słownika**:
   ```python
   idx_to_word = {v: k for k, v in word_to_idx.items()}
   ```
   - Tworzony jest odwrotny słownik, który mapuje indeksy na słowa. To pozwala na zamianę indeksów z powrotem na tekst.

5. **Podgląd pierwszych 10 elementów słownika**:
   ```python
   list(idx_to_word.items())[:10]
   ```
   - Wyświetlana jest pierwsza dziesiątka par (indeks, słowo) z odwróconego słownika, żeby zobaczyć, jak indeksy są przypisane do słów.

6. **Wyświetlenie pierwszego zdania w `train_data`**:
   ```python
   print(' '.join(idx_to_word[idx] for idx in train_data[0]))
   ```
   - `train_data[0]` to pierwsza sekwencja (zdanie) z danych treningowych, która jest zakodowana jako sekwencja indeksów.
   - Każdy indeks jest tłumaczony na odpowiadające mu słowo z pomocą słownika `idx_to_word`, a następnie słowa są łączone w zdanie za pomocą funkcji `' '.join()`.

### Przykład działania:
Jeśli pierwszy wiersz w `train_data[0]` zawiera indeksy np. `[1, 25, 43, 7, 2]`, kod zamieni te indeksy na słowa:
- `1` to `'<START>'`
- `25` to np. `"apple"`
- `43` to np. `"is"`
- `7` to np. `"good"`
- `2` to `'<UNK>'` (nieznane słowo)

Wynikowy tekst, który zostanie wyświetlony, mógłby wyglądać tak:
```
<START> apple is good <UNK>
```

Ten kod pomaga w tłumaczeniu sekwencji indeksów z powrotem na czytelny tekst, co jest użyteczne podczas interpretacji wyników modelu oraz do weryfikacji poprawności przetwarzania danych.

In [6]:
train_labels[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [7]:
def multi_hot_sequences(sequences, dimension):
    results = np.zeros((len(sequences), dimension))
    for i, word_indices in enumerate(sequences):
        results[i, word_indices] = 1.0
    return results

train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
train_data.shape

(25000, 10000)

Ten kod w Pythonie tworzy **"multi-hot encoding"** dla sekwencji danych, prawdopodobnie reprezentujących indeksy słów w tekście, np. w ramach przetwarzania języka naturalnego (NLP). Wyjaśnię krok po kroku:

### Funkcja `multi_hot_sequences`

#### Argumenty:
- `sequences`: Lista sekwencji (czyli lista list), gdzie każda podlista zawiera indeksy słów.
- `dimension`: Całkowita liczba możliwych słów (rozmiar słownika), czyli ile unikalnych słów może wystąpić w tekście.

#### Działanie:
1. **Tworzenie macierzy `results`:**
   ```python
   results = np.zeros((len(sequences), dimension))
   ```
   - Funkcja tworzy macierz o wymiarach `(liczba sekwencji, liczba możliwych słów)`, czyli np. `(10000, 5000)`, gdzie `10000` to liczba zdań (sekwencji), a `5000` to liczba możliwych słów.
   - Na początku macierz jest wypełniona zerami.

2. **Iteracja po sekwencjach i wypełnianie macierzy:**
   ```python
   for i, word_indices in enumerate(sequences):
       results[i, word_indices] = 1.0
   ```
   - `enumerate(sequences)` iteruje po każdej sekwencji w `sequences`.
   - `i` to indeks sekwencji, a `word_indices` to lista indeksów słów dla danej sekwencji.
   - Dla każdego zdania (sekwencji), funkcja ustawia `1.0` w miejscach odpowiadających indeksom słów w tej sekwencji. W ten sposób dla danego zdania powstaje wektor, który ma wartość `1` tam, gdzie dane słowo występuje, a `0`, gdzie go nie ma.

3. **Zwrócenie wyniku:**
   ```python
   return results
   ```
   - Zwracana jest macierz `results`, która zawiera multi-hot encoding dla każdej sekwencji.

### Wywołanie funkcji:
```python
train_data = multi_hot_sequences(train_data, dimension=NUM_WORDS)
test_data = multi_hot_sequences(test_data, dimension=NUM_WORDS)
```
- Funkcja `multi_hot_sequences` jest używana, aby zamienić dane treningowe i testowe (sekwencje indeksów słów) na ich odpowiednie **multi-hot encodingi**.
- `NUM_WORDS` to liczba słów w słowniku (czyli wartość `dimension`).

### Ostatnia linia:
```python
train_data.shape
```
- `train_data.shape` zwraca wymiary macierzy `train_data`, co pozwala sprawdzić, ile jest wierszy (sekwencji) i ile kolumn (słów).

### Przykład:
Jeśli `train_data` wygląda tak:
```python
train_data = [[1, 3], [2, 4, 5]]
dimension = 6
```

Macierz wynikowa po zakodowaniu będzie wyglądała tak:
```
[[0, 1, 0, 1, 0, 0],   # W pierwszym zdaniu występują słowa o indeksach 1 i 3
 [0, 0, 1, 0, 1, 1]]   # W drugim zdaniu występują słowa o indeksach 2, 4 i 5
```

Każdy wiersz reprezentuje jedno zdanie, a kolumny odpowiadają wszystkim możliwym słowom w słowniku. Kolumna ma wartość `1`, gdy dane słowo występuje w zdaniu, a `0`, gdy nie występuje.

In [8]:
test_data.shape

(25000, 10000)